In [ ]:
import pandas as pd
%matplotlib inline
df = pd.read_csv('./wifi/application.csv', usecols=[0,1], engine='python', skipfooter=0)
df.plot()

In [ ]:
import numpy as np
import pandas as pd
import tflearn
import matplotlib.pyplot as plt
dataframe = pd.read_csv('./wifi/application.csv',
        usecols=[1],
        engine='python',
        skipfooter=0)
dataset = dataframe.values
dataset = dataset.astype('float32')

In [ ]:
dataset -= np.min(np.abs(dataset))
dataset /= np.max(np.abs(dataset))

In [ ]:
def create_dataset(dataset, steps_of_history, steps_in_future):
    X, Y = [], []
    for i in range(0, len(dataset)-steps_of_history, steps_in_future):
        X.append(dataset[i:i+steps_of_history])
        Y.append(dataset[i + steps_of_history])
    X = np.reshape(np.array(X), [-1, steps_of_history, 1])
    Y = np.reshape(np.array(Y), [-1, 1])
    return X, Y

def split_data(x, y, test_size=0.1):
    pos = round(len(x) * (1 - test_size))
    trainX, trainY = x[:pos], y[:pos]
    testX, testY   = x[pos:], y[pos:]
    return trainX, trainY, testX, testY

steps_of_history = 1
steps_in_future = 1

X, Y = create_dataset(dataset, steps_of_history, steps_in_future)
trainX, trainY, testX, testY = split_data(X, Y, 0.1)

In [ ]:
net = tflearn.input_data(shape=[None, steps_of_history, 1])
net = tflearn.lstm(net, n_units=6)
# net = tflearn.gru(net, n_units=6, return_seq=True)
# net = tflearn.gru(net, n_units=6)
net = tflearn.fully_connected(net, 1, activation='linear')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='mean_square')

model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, validation_set=0.1, batch_size=1, n_epoch=100)

In [ ]:
testX2 = np.reshape(np.array(testX), [-1, 1, 1])

In [ ]:
score = model.evaluate(testX2, testY)
score

In [ ]:
train_predict = model.predict(trainX)
test_predict = model.predict(testX)

train_predict_plot = np.empty_like(dataset)
train_predict_plot[:, :] = np.nan
train_predict_plot[steps_of_history:len(train_predict)+steps_of_history, :] = train_predict

test_predict_plot = np.empty_like(dataset)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict)+steps_of_history:len(dataset), :] = test_predict

plt.figure(figsize=(20, 10))
plt.title('History={} Future={}'.format(steps_of_history, steps_in_future))
plt.plot(dataset, color='blue', label="dataset")
plt.plot(train_predict_plot, color='green', label="train_predict")
plt.plot(test_predict_plot, color='red', label="test_predict")
plt.legend(ncol=3, mode="expand")
plt.savefig('wifi.png')

In [ ]:
train_predict

In [ ]:
dataset

In [ ]:
# 平均二乗平方根誤差
def rmse(y_pred, y_true):
    return np.sqrt(((y_true - y_pred) ** 2).mean())

def rmsle(y_pred, y_true):
    return np.sqrt(np.square(np.log(y_true + 1) - np.log(y_pred + 1)).mean())

# 平均絶対誤差
def mae(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred)))

# 平均絶対誤差率
def mape(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
trainX = np.reshape(np.array(trainX), [-1, 1])
testX = np.reshape(np.array(testX), [-1, 1])
delta = 1e-10

----
### 訓練データ

In [ ]:
len(trainX)

In [ ]:
len(train_predict)

In [ ]:
rmse_train = rmse(train_predict, trainX)

In [ ]:
mae_train = mae(train_predict, trainX)

In [ ]:
mape_train = mape(train_predict[1:], trainX[1:])

----
### テストデータ

In [ ]:
rmse_test = rmse(test_predict, testX)

In [ ]:
mae_test = mae(test_predict, testX)

In [ ]:
mape_test = mape(test_predict, testX)

----
##### テストデータ 直近のみ

In [ ]:
test_predict_last05 = test_predict[1:7]
testX_last05 = testX[1:7]
test_predict_last10 = test_predict[1:10]
testX_last10 = testX[1:10]

In [ ]:
rmse_test_last05 = rmse(test_predict_last05, testX_last05)
rmse_test_last10 = rmse(test_predict_last10, testX_last10)

In [ ]:
mae_test_last05 = mae(test_predict_last05, testX_last05)
mae_test_last10 = mae(test_predict_last10, testX_last10)

In [ ]:
mape_test_last05 = mape(test_predict_last05, testX_last05)
mape_test_last10 = mape(test_predict_last10, testX_last10)

----

In [ ]:
print(rmse_train, rmse_test, rmse_test_last10, rmse_test_last05)
print(mae_train, mae_test, mae_test_last10, mae_test_last05)
print(mape_train, mape_test, mape_test_last10, mape_test_last05)

In [ ]:
for m in range(0, 3): # 3 months
    m_p = np.concatenate((test_predict[m*4*7:(m+1)*4*7]))
    m_t = np.concatenate((testX[m*4*7:(m+1)*4*7]))
    print('month-%02d' % (m+1), ':', mape(m_p, m_t))
for w in range(0, 3*4): # 3 months * 4 weeks
    w_p = np.concatenate((test_predict[w*7:(w+1)*7]))
    w_t = np.concatenate((testX[w*7:(w+1)*7]))
    print('week-%02d' % (w+1), ':', mape(w_p, w_t))
for d in range(3*4*7): # 3 months * 4 weeks * 7 days
    print('day-%02d' % (d+1), ':', mape(test_predict[d], testX[d])) 